In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

df1 = pd.read_csv('13oct.csv')              # change file name for raw data
df2 = pd.read_csv('Spectral Temps.csv')     # change name to ProcessedData
merged_df = df1.join(df2, how = 'right', index = False)

merged_df.to_csv('AllData13oct.csv', index = False)  # the index False should get rid of the first column of indices
master = pd.read_csv('AllData13oct.csv')
display(master)

,Date,comments,Time,Inj (A),Extr (A),Cent (A),Sext (A),JYB Bmin (T),JYB Ts,Inj (T),...,Sum of counts,Run #,Spectral Temp,Error,Sum Counts,Livetimes,Coeff A,Coeff B,Coeff C,Coeff D
0,10/13/25,replica of 7/16/15 and 2/3/16,3:52 PM,122,105,102,270,0.296,43.790150,2.23,...,NaN,Run01,36.103520,2.915319,2.306673,300.000000,1.77789,-0.016349,0.000053,-5.623966e-08
1,10/13/25,"assume taken at 1000 W, not sure",4:00 PM,122,105,98,270,0.324,46.563801,2.24,...,NaN,Run02,32.241373,2.357919,3.648542,300.000000,1.77789,-0.016349,0.000053,-5.623966e-08
2,10/13/25,NaN,4:07 PM,122,105,94,270,0.351,55.043533,2.25,...,NaN,Run03,29.695920,1.832329,4.851508,300.000000,1.77789,-0.016349,0.000053,-5.623966e-08
3,10/13/25,NaN,4:15:52 PM,122,105,88,270,0.393,61.617901,2.25,...,NaN,Run04,30.433303,1.975460,7.052675,301.989000,1.77789,-0.016349,0.000053,-5.623966e-08
4,10/13/25,NaN,4:27:32 PM,122,105,120,270,0.170,27.167209,2.20,...,NaN,Run05,135.984473,24.742482,0.323849,240.000000,1.77789,-0.016349,0.000053,-5.623966e-08
5,10/13/25,NaN,4:33:43 PM,122,105,125,270,0.138,26.899981,2.19,...,NaN,Run06,158.796360,35.350256,0.336856,240.000000,1.77789,-0.016349,0.000053,-5.623966e-08
6,10/13/25,replica of 11/2/15,4:47:27 PM,124,108,110,270,0.332,34.206534,2.27,...,NaN,Run07,90.202922,16.465366,0.833925,240.000000,1.77789,-0.016349,0.000053,-5.623966e-08
7,10/13/25,all done at 1000 W,4:51:53 PM,122,105,102,270,0.296,38.517712,2.23,...,NaN,Run08,36.978436,3.061655,2.032306,240.000000,1.77789,-0.016349,0.000053,-5.623966e-08
8,10/13/25,NaN,5:07:37 PM,120,103,105,270,0.258,31.183123,2.17,...,NaN,Run09,116.940432,27.300758,0.770898,240.000000,1.77789,-0.016349,0.000053,-5.623966e-08
9,10/13/25,repeated after changing settings to match Jani...,5:15 PM,120,103,105,270,0.258,31.183123,2.17,...,NaN,Run10,89.505823,20.742907,2.157290,299.445939,1.77789,-0.016349,0.000053,-5.623966e-08


In [9]:
plt.scatter(master['Input Counts'], master['Ts'])
plt.xlabel('Counts')
plt.ylabel('Spectral Temperature (eV)')
#plt.scatter(df['Bmin (Panelview)'], color = 'k', df['Ts'], color = 'red')
plt.title('Counts vs Ts')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
plt.scatter(master['Bmin (Panelview)'], master['Ts'])
plt.xlabel('Bmin (T)')
plt.ylabel('Spectral Temperature (eV)')
#plt.scatter(df['Bmin (Panelview)'], color = 'k', df['Ts'], color = 'red')
plt.title('Bmin vs Ts')
plt.show()

In [ ]:
plt.scatter(df['Total Power Ideal'], df['Ts'])
plt.xlabel('Total Power (W)')
plt.ylabel('Spectral Temperature (eV)')
plt.title('Power vs Ts')
plt.show()

In [ ]:
plt.scatter(df['Inj Pres (-7)'], df['Ts'])
plt.xlabel('Injection Pressure (uTorr)')
plt.ylabel('Spectral Temperature (eV)')
plt.title('Injection Pressure vs Ts')
plt.show()

In [ ]:
plt.scatter(df['Puller I'], df['Ts'])
plt.xlabel('Puller I')
plt.ylabel('Spectral Temperature (eV)')
plt.title('Puller Current vs Ts')
plt.show()

#see what effect high puller current has, on CSDs, count rate, etc

In [ ]:
plt.scatter(df['Sext (T)'], df['Ts'])
plt.xlabel('Brad (T)')
plt.ylabel('Spectral Temperature (eV)')
plt.title('Brad vs Ts')
plt.show()

In [ ]:
plt.scatter(df['Sext (T)'] / df['Extr (T)'], df['Ts'])
plt.xlabel('Brad/Bextr')
plt.ylabel('Spectral Temperature (eV)')
plt.title('Brad/Bextr vs Ts')
plt.show()

#can you do math with dataframes

In [ ]:
plt.scatter(df['28 GHz'] / df['18 GHz'], df['Ts'])
plt.xlabel('28 / 18 GHz')
plt.ylabel('Spectral Temperature (eV)')
plt.title('28 / 18 GHz Ratio vs Ts')
plt.show()

In [ ]:
plt.scatter(df['18 GHz'], df['Ts'])
plt.scatter(df['28 GHz'], df['Ts'])
plt.xlabel('Power (W)')
plt.ylabel('Spectral Temperature (eV)')
plt.title('Power (W) vs Ts')
plt.show()

In [ ]:
all3 = df[(df['14 GHz Ideal'] == 500) & (df['18GHz Ideal'] == 500) & (df['28GHz Ideal'] == 500)]
sub_1828G = df[(df['14 GHz Ideal'] == 0) & (df['18GHz Ideal'] == 500) & (df['28GHz Ideal'] == 500)]
sub_1428G = df[(df['14 GHz Ideal'] == 500) & (df['18GHz Ideal'] == 0) & (df['28GHz Ideal'] == 500)]
sub_28G = df[(df['14 GHz Ideal'] == 0) & (df['18GHz Ideal'] == 0) & (df['28GHz Ideal'] == 500)]

plt.plot(sub_28G['Bmin (Panelview)'], sub_28G['Ts'], marker = 'o', label = '28 GHz only (500 W)')
plt.plot(sub_1828G['Bmin (Panelview)'], sub_1828G['Ts'], marker = 'o', label = '18 + 28 GHz (1000 W)')
plt.plot(sub_1428G['Bmin (Panelview)'], sub_1428G['Ts'], marker = 'o', label = '14 + 28 GHz (1000 W)')
plt.plot(all3['Bmin (Panelview)'], all3['Ts'], marker = 'o', label = '14 + 18 + 28 GHz (1500 W)')

plt.xlabel('Bmin (T)')
plt.ylabel('Ts')
plt.title('Bmin vs Ts')
plt.ylim(0,100)
plt.legend()
plt.show()

In [ ]:
plt.plot(sub_1828G['Bmin (Panelview)'], sub_1828G['Ts'], marker = 'o', label = '18 + 28 GHz (1000 W)')
plt.plot(sub_1428G['Bmin (Panelview)'], sub_1428G['Ts'], marker = 'o', label = '14 + 28 GHz (1000 W)')

plt.xlabel('Bmin (T)')
plt.ylabel('Ts')
plt.title('Bmin vs Ts')
plt.legend()
plt.show()